# 모듈 import

In [1]:
!pip install selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
pd.set_option('display.max_columns', None) # 데이터프레임 컬럼 display

import warnings
warnings.filterwarnings('ignore') # 경고 무시

Defaulting to user installation because normal site-packages is not writeable


<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# 한 경기의 정보 가져오기
- 크롤링
- 데이터 정리

#### 크롤링
- selenium : 웹 제어 (클릭)
- BeautifulSoup : html에서 값 읽기

In [2]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# 크롬 브라우저
driver = webdriver.Chrome()
# url 접속
url = "https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=017&g_part=201&r_round=1&g_num=2&"
driver.get(url)
# 1초 대기
time.sleep(1)

#--------------------- table 태그 읽기 - 팀 별 1 ~ 5 세트 결과 ---------------------#
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all('table')
set_result = pd.read_html(str(table))[1] # 1 ~ 5 세트 별 득점 결과
display(set_result)

#--------------------- 날짜 정보 읽기 ---------------------#
date_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > thead > tr > th'
stadium_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > tfoot > tr:nth-child(1) > td:nth-child(2) > span'
referee_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > tfoot > tr:nth-child(2) > td > span'
date = soup.select_one(date_selector).get_text()
display(date)
date = date.replace('\xa0', '').split('/')[0].strip().split()
date = pd.to_datetime(date[0][:-1] + '/' + date[1][:-1] + '/' + date[2][:-1])
stadium = soup.select_one(stadium_selector).get_text() #경기장 정보
stadium_name = list(stadium.split('/')[0].split())[0] #경기장 이름
stadium_crowd = int(list(stadium.split('/')[2].split())[1][:-1]) #관중수

#
# 첫 번째 팀 정보 읽기
#

#--------------------- 버튼 클릭 - 선수 기록 ---------------------#
x_path = '//*[@id="wrp_content"]/article[2]/ul/li[2]/a/span' # 선수 기록 버튼
cursor = driver.find_element(By.XPATH, x_path)
cursor.click()
time.sleep(1)

#--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all('table')

team1 = pd.read_html(str(table))[3] # 명단 table
team1_info = pd.read_html(str(table))[4] # 출전 세트, 득점, 공격 종합 table
team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
team1 = pd.concat([team1, team1_info], axis = 1)

#--------------------- 나머지 기록 읽기 ---------------------#
for _ in range(4) :
    #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
    x_path = '//*[@id="team1"]/div/div[1]/a[2]' # 다음 기록 버튼
    cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)
    
    
    #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    
    team1_info = pd.read_html(str(table))[4] # 다음 기록
    team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
    team1 = pd.concat([team1, team1_info], axis = 1)
display(team1)


#
# 두 번째 팀 정보 읽기
#

#--------------------- 버튼 클릭 - 두 번째 팀 ---------------------#
x_path = '//*[@id="tab2"]/div[2]/ul/li[2]/a/span' # 두 번째 팀 버튼
cursor = driver.find_element(By.XPATH,x_path)
cursor.click()
time.sleep(1)

#--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all('table')

team2 = pd.read_html(str(table))[5] # 명단 table
team2_info = pd.read_html(str(table))[6] # 출전 세트, 득점, 공격 종합 table
team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
team2 = pd.concat([team2, team2_info], axis = 1)

#--------------------- 나머지 기록 읽기 ---------------------#
for _ in range(4) :
    #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
    x_path = '//*[@id="team2"]/div/div[1]/a[2]' # 다음 기록 버튼
    cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)
    
    
    #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    
    team2_info = pd.read_html(str(table))[6] # 다음 기록
    team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
    team2 = pd.concat([team2, team2_info], axis = 1)
display(team2)

,여자부,1SET,2SET,3SET,4SET,5SET,TOTAL
0,현대건설,19,25,25,20,15,104
1,GS칼텍스,25,22,21,25,9,102
2,경기시간,26 m,33 m,26 m,28 m,15 m,2h08m


' 2020년\xa0 10월\xa0 17일 (토) / 제 2경기'

,No.,이름,출전세트_1set,출전세트_2set,출전세트_3set,출전세트_4set,출전세트_5set,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실
0,10,루소 (L),O,O,O,O,O,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6
1,13,정지윤 (C),O,O,O,O,O,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2
2,14,양효진 (C),O,O,O,O,O,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3
3,17,고예림 (L),O,O,O,O,O,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2
4,15,황민경 (L),O,O,O,O,NaN,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1
5,12,이다현 (C),17,17,17,17,O,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1
6,3,김다인 (S),O,O,O,O,O,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6
7,8,김연견 (Li),LI,LI,LI,LI,LI,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0
8,11,김주하 (Li),LI,LI,LI,LI,LI,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0
9,6,이나연 (S),NaN,NaN,NaN,NaN,13,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,4,3,0,3.0,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0


,No.,이름,출전세트_1set,출전세트_2set,출전세트_3set,출전세트_4set,출전세트_5set,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실
0,13,러츠 (R),O,O,O,O,O,33,66,26,4,6,39.39,42.31,37,12,2,4,32.43,46.25,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,14,7,0,2,50.0,87.50,0,0,0,0,0.00,0.0,15,7,2,0,46.67,30.0,13,2,3,0.40,12.75,9,5,4,0,1.00,9.57,5,0,1,0.0,3.16,0,0,0,0.00,0.00,15,5,3,2,0,1.0,20.83,2,0,10
1,10,강소휘 (L),O,O,O,O,O,21,37,19,2,0,51.35,23.72,15,8,0,0,53.33,18.75,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,1,0,0,0,0.0,6.25,0,0,0,0,0.00,0.0,21,11,2,0,52.38,42.0,15,2,4,0.40,14.71,12,10,2,0,2.00,12.77,5,0,0,0.0,3.16,26,14,0,53.85,27.66,8,0,1,4,0,0.0,11.11,1,0,4
2,1,이소영 (L),O,O,19,O,O,9,29,7,4,2,24.14,18.59,17,5,1,0,29.41,21.25,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,1,0,0,0,0.0,6.25,0,0,0,0,0.00,0.0,11,2,3,2,18.18,22.0,11,1,0,0.20,10.78,11,8,3,0,1.60,11.70,1,1,0,0.2,0.63,26,8,0,30.77,27.66,11,1,2,3,0,0.2,15.28,1,0,2
3,9,한수지 (C),O,O,O,O,O,7,5,2,0,0,40.00,3.21,2,1,0,0,50.00,2.50,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.00,3,1,0,0,33.33,30.0,0,0,0,0,0.00,0.0,11,0,0,0.00,10.78,7,7,0,0,1.40,7.45,4,2,0,0.4,2.53,1,0,0,0.00,1.06,23,5,10,6,0,1.0,31.94,3,0,0
4,7,안혜진 (S),O,O,O,O,O,6,1,1,0,0,100.00,0.64,1,1,0,0,100.00,1.25,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.0,26,4,1,0.80,25.49,19,17,2,0,3.40,20.21,120,44,1,8.8,75.95,0,0,0,0.00,0.00,6,1,1,4,0,0.2,8.33,0,0,2
5,16,김유리 (C),O,O,O,O,O,3,8,3,0,0,37.50,5.13,1,0,0,0,0.00,1.25,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.00,7,3,0,0,42.86,70.0,0,0,0,0,0.00,0.0,5,0,0,0.00,4.90,4,2,2,0,0.40,4.26,2,0,0,0.0,1.27,0,0,0,0.00,0.00,6,0,2,4,0,0.0,8.33,2,0,0
6,18,권민지 (L),NaN,7,16,NaN,NaN,1,5,0,1,1,0.00,3.21,3,0,0,1,0.00,3.75,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.0,2,0,1,0,0.00,4.0,4,1,2,0.50,3.92,2,2,0,0,1.00,2.13,2,1,0,0.5,1.27,0,0,0,0.00,0.00,2,0,0,1,0,0.0,2.78,0,0,3
7,19,유서연 (R),1,NaN,O,NaN,1,1,4,0,0,2,0.00,2.56,3,0,0,1,0.00,3.75,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.0,1,0,0,1,0.00,2.0,4,1,0,0.33,3.92,3,1,2,0,0.33,3.19,0,0,0,0.0,0.00,10,4,0,40.00,10.64,1,0,0,1,0,0.0,1.39,0,0,2
8,6,이원정 (S),NaN,13,7,NaN,NaN,0,1,0,0,0,0.00,0.64,1,0,0,0,0.00,1.25,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.0,2,0,0,0.00,1.96,2,2,0,0,1.00,2.13,8,4,0,2.0,5.06,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0
9,8,김해빈 (L),NaN,NaN,NaN,9,9,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.0,6,0,0,0.00,5.88,1,0,1,0,0.00,1.06,0,0,0,0.0,0.00,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0


<br/>
<br/>
<br/>
<br/>

#### 데이터 정리
- team1, team2 동시에 처리

In [3]:
# No. 열 삭제
team1 = team1.drop('No.', axis = 1)
team2 = team2.drop('No.', axis = 1)
team1

,이름,출전세트_1set,출전세트_2set,출전세트_3set,출전세트_4set,출전세트_5set,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실
0,루소 (L),O,O,O,O,O,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6
1,정지윤 (C),O,O,O,O,O,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2
2,양효진 (C),O,O,O,O,O,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3
3,고예림 (L),O,O,O,O,O,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2
4,황민경 (L),O,O,O,O,NaN,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1
5,이다현 (C),17,17,17,17,O,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1
6,김다인 (S),O,O,O,O,O,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6
7,김연견 (Li),LI,LI,LI,LI,LI,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0
8,김주하 (Li),LI,LI,LI,LI,LI,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0
9,이나연 (S),NaN,NaN,NaN,NaN,13,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,4,3,0,3.0,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0


In [4]:
# 3경기 이상 출전한 선수만 유지 (출전 세트 결측치가 3개 이상인 선수 제거)
subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

# thresh = 3 : 정상 값(Not NaN)이 3개 이상만 유지
team1 = team1.dropna(subset = subset, thresh = 3)
team2 = team2.dropna(subset = subset, thresh = 3)
team1

,이름,출전세트_1set,출전세트_2set,출전세트_3set,출전세트_4set,출전세트_5set,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실
0,루소 (L),O,O,O,O,O,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6
1,정지윤 (C),O,O,O,O,O,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2
2,양효진 (C),O,O,O,O,O,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3
3,고예림 (L),O,O,O,O,O,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2
4,황민경 (L),O,O,O,O,NaN,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1
5,이다현 (C),17,17,17,17,O,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1
6,김다인 (S),O,O,O,O,O,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6
7,김연견 (Li),LI,LI,LI,LI,LI,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0
8,김주하 (Li),LI,LI,LI,LI,LI,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0


In [5]:
# 스타팅멤버 표시 (3세트 이상 스타팅멤버라면 1 아니면 0)
team1['스타팅멤버'] = 0
team2['스타팅멤버'] = 0

team1.loc[(team1.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
team2.loc[(team2.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
team1

,이름,출전세트_1set,출전세트_2set,출전세트_3set,출전세트_4set,출전세트_5set,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,스타팅멤버
0,루소 (L),O,O,O,O,O,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6,1
1,정지윤 (C),O,O,O,O,O,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2,1
2,양효진 (C),O,O,O,O,O,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3,1
3,고예림 (L),O,O,O,O,O,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2,1
4,황민경 (L),O,O,O,O,NaN,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1,1
5,이다현 (C),17,17,17,17,O,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1,0
6,김다인 (S),O,O,O,O,O,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6,1
7,김연견 (Li),LI,LI,LI,LI,LI,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0,0
8,김주하 (Li),LI,LI,LI,LI,LI,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0,0


In [6]:
# 출전 세트 열 삭제
subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

team1 = team1.drop(subset, axis = 1)
team2 = team2.drop(subset, axis = 1)
team1

,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,스타팅멤버
0,루소 (L),28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6,1
1,정지윤 (C),21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2,1
2,양효진 (C),18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3,1
3,고예림 (L),8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2,1
4,황민경 (L),2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1,1
5,이다현 (C),2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1,0
6,김다인 (S),1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6,1
7,김연견 (Li),0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0,0
8,김주하 (Li),0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0,0


In [7]:
# 이름과 포지션 분리하기
# "이름" 열 : 이름 정보
# "포지션" 열 : 포지션 정보

team1['포지션'] = team1['이름'].str.split().str[1].str.strip('()')
team1['이름'] = team1['이름'].str.split().str[0]

team2['포지션'] = team2['이름'].str.split().str[1].str.strip('()')
team2['이름'] = team2['이름'].str.split().str[0]
team1

,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,스타팅멤버,포지션
0,루소,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6,1,L
1,정지윤,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2,1,C
2,양효진,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3,1,C
3,고예림,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2,1,L
4,황민경,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1,1,L
5,이다현,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1,0,C
6,김다인,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6,1,S
7,김연견,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0,0,Li
8,김주하,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0,0,Li


In [8]:
# 선수 별 득점 점유율 구하기 (개인 별 득점을 팀 전체의 득점으로 나누기)

team1_total_score = team1['득점_득점'].sum()
team2_total_score = team2['득점_득점'].sum()
team1['득점점유율'] = team1['득점_득점'] / team1_total_score
team2['득점점유율'] = team2['득점_득점'] / team2_total_score
team1

,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,스타팅멤버,포지션,득점점유율
0,루소,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6,1,L,0.3500
1,정지윤,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2,1,C,0.2625
2,양효진,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3,1,C,0.2250
3,고예림,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2,1,L,0.1000
4,황민경,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1,1,L,0.0250
5,이다현,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1,0,C,0.0250
6,김다인,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6,1,S,0.0125
7,김연견,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0,0,Li,0.0000
8,김주하,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0,0,Li,0.0000


In [9]:
team1['경기장'] = stadium_name
team1['관중수'] = stadium_crowd
team2['경기장'] = stadium_name
team2['관중수'] = stadium_crowd

In [10]:
columns = [
  '이름',
  '경기장',
  '관중수',
  '득점_득점',
  '공격종합_시도',
  '공격종합_성공',
  '공격종합_공격차단',
  '공격종합_범실',
  '공격종합_성공률',
  '공격종합_점유율',
  '오픈_시도',
  '오픈_성공',
  '오픈_공격차단',
  '오픈_범실',
  '오픈_성공률',
  '오픈_점유율',
  '시간차_시도',
  '시간차_성공',
  '시간차_공격차단',
  '시간차_범실',
  '시간차_성공률',
  '시간차_점유율',
  '이동_시도',
  '이동_성공',
  '이동_공격차단',
  '이동_범실',
  '이동_성공률',
  '이동_점유율',
  '후위_시도',
  '후위_성공',
  '후위_공격차단',
  '후위_범실',
  '후위_성공률',
  '후위_점유율',
  '속공_시도',
  '속공_성공',
  '속공_공격차단',
  '속공_범실',
  '속공_성공률',
  '속공_점유율',
  '퀵오픈_시도',
  '퀵오픈_성공',
  '퀵오픈_공격차단',
  '퀵오픈_범실',
  '퀵오픈_성공률',
  '퀵오픈_점유율',
  '서브_시도',
  '서브_성공',
  '서브_범실',
  '서브_성공률',
  '서브_점유율',
  '디그_시도',
  '디그_성공',
  '디그_실패',
  '디그_범실',
  '디그_세트당',
  '디그_점유율',
  '세트_시도',
  '세트_성공',
  '세트_범실',
  '세트_세트당',
  '세트_점유율',
  '리시브_시도',
  '리시브_정확',
  '리시브_실패',
  '리시브_세트당',
  '리시브_점유율',
  '블로킹_시도',
  '블로킹_성공',
  '블로킹_유효블락',
  '블로킹_실패',
  '블로킹_범실',
  '블로킹_세트당',
  '블로킹_점유율',
  '블로킹_어시스트',
  '벌칙_벌칙',
  '범실_범실',
  '포지션',
  '득점점유율',
  '스타팅멤버',
];


team1 = team1[columns]
team2 = team2[columns]
team1

,이름,경기장,관중수,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,루소,수원체육관,0,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6,L,0.3500,1
1,정지윤,수원체육관,0,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2,C,0.2625,1
2,양효진,수원체육관,0,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3,C,0.2250,1
3,고예림,수원체육관,0,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2,L,0.1000,1
4,황민경,수원체육관,0,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1,L,0.0250,1
5,이다현,수원체육관,0,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1,C,0.0250,0
6,김다인,수원체육관,0,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6,S,0.0125,1
7,김연견,수원체육관,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0,Li,0.0000,0
8,김주하,수원체육관,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0,Li,0.0000,0


In [11]:
# 경기날짜, 팀명, 결과 정보 추가
team1['팀명'] = set_result.iloc[0,0]
team2['팀명'] = set_result.iloc[1,0]
team1['결과'] = 1 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 0
team2['결과'] = 0 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 1
team1['경기날짜'] = date
team2['경기날짜'] = date


# 컬럼 순서 변경
team1 = team1[['팀명', '경기장', '관중수', '경기날짜', '결과'] + columns]
team2 = team2[['팀명', '경기장', '관중수', '경기날짜', '결과'] + columns]
team1

,팀명,경기장,관중수,경기날짜,결과,이름,경기장,관중수,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,현대건설,수원체육관,0,2020-10-17,1,루소,수원체육관,0,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.0,0.00,15,5,0,1,33.33,51.72,16,2,3,0.4,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6,L,0.3500,1
1,현대건설,수원체육관,0,2020-10-17,1,정지윤,수원체육관,0,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.0,27.78,3,2,0,0,66.67,10.34,9,0,1,0.0,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2,C,0.2625,1
2,현대건설,수원체육관,0,2020-10-17,1,양효진,수원체육관,0,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.0,66.67,0,0,0,0,0.00,0.00,21,0,1,0.0,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3,C,0.2250,1
3,현대건설,수원체육관,0,2020-10-17,1,고예림,수원체육관,0,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.0,0.00,8,3,1,2,37.50,27.59,15,0,0,0.0,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2,L,0.1000,1
4,현대건설,수원체육관,0,2020-10-17,1,황민경,수원체육관,0,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,3,0,0,0,0.00,10.34,15,0,1,0.0,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1,L,0.0250,1
5,현대건설,수원체육관,0,2020-10-17,1,이다현,수원체육관,0,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.0,5.56,0,0,0,0,0.00,0.00,2,0,1,0.0,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1,C,0.0250,0
6,현대건설,수원체육관,0,2020-10-17,1,김다인,수원체육관,0,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,22,0,3,0.0,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6,S,0.0125,1
7,현대건설,수원체육관,0,2020-10-17,1,김연견,수원체육관,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0,Li,0.0000,0
8,현대건설,수원체육관,0,2020-10-17,1,김주하,수원체육관,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,0,0,0,0.0,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0,Li,0.0000,0


In [12]:
# 연결
team = pd.concat([team1, team2])
team

,팀명,경기장,관중수,경기날짜,결과,이름,경기장,관중수,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,현대건설,수원체육관,0,2020-10-17,1,루소,수원체육관,0,28,54,22,3,3,40.74,35.29,27,10,3,2,37.04,29.35,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,12,7,0,0,58.33,92.31,0,0,0,0,0.00,0.00,15,5,0,1,33.33,51.72,16,2,3,0.40,15.38,11,8,3,0,1.60,10.28,3,0,0,0.0,1.89,12,2,3,0.00,13.19,15,4,2,5,0,0.8,19.48,2,0,6,L,0.3500,1
1,현대건설,수원체육관,0,2020-10-17,1,정지윤,수원체육관,0,21,34,20,3,1,58.82,22.22,26,16,2,1,61.54,28.26,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,5,2,1,0,40.00,27.78,3,2,0,0,66.67,10.34,9,0,1,0.00,8.65,10,8,2,0,1.60,9.35,2,1,0,0.2,1.26,0,0,0,0.00,0.00,15,1,5,4,0,0.2,19.48,0,0,2,C,0.2625,1
2,현대건설,수원체육관,0,2020-10-17,1,양효진,수원체육관,0,18,32,15,2,2,46.88,20.92,20,9,1,0,45.00,21.74,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,12,6,1,2,50.00,66.67,0,0,0,0,0.00,0.00,21,0,1,0.00,20.19,10,7,3,0,1.40,9.35,3,2,0,0.4,1.89,3,1,0,33.33,3.30,13,3,2,5,0,0.6,16.88,5,0,3,C,0.2250,1
3,현대건설,수원체육관,0,2020-10-17,1,고예림,수원체육관,0,8,19,6,3,2,31.58,12.42,9,2,2,0,22.22,9.78,1,1,0,0,100.0,100.0,0,0,0,0,0.0,0.0,1,0,0,0,0.00,7.69,0,0,0,0,0.00,0.00,8,3,1,2,37.50,27.59,15,0,0,0.00,14.42,16,14,2,0,2.80,14.95,7,2,0,0.4,4.40,21,10,0,47.62,23.08,8,2,3,1,0,0.4,10.39,0,0,2,L,0.1000,1
4,현대건설,수원체육관,0,2020-10-17,1,황민경,수원체육관,0,2,12,2,1,0,16.67,7.84,9,2,1,0,22.22,9.78,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,3,0,0,0,0.00,10.34,15,0,1,0.00,14.42,15,13,2,0,3.25,14.02,6,2,0,0.5,3.77,33,14,5,27.27,36.26,10,0,3,6,0,0.0,12.99,1,0,1,L,0.0250,1
5,현대건설,수원체육관,0,2020-10-17,1,이다현,수원체육관,0,2,2,2,0,0,100.00,1.31,1,1,0,0,100.00,1.09,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,1,1,0,0,100.00,5.56,0,0,0,0,0.00,0.00,2,0,1,0.00,1.92,0,0,0,0,0.00,0.00,1,1,0,0.2,0.63,0,0,0,0.00,0.00,7,0,4,2,0,0.0,9.09,1,0,1,C,0.0250,0
6,현대건설,수원체육관,0,2020-10-17,1,김다인,수원체육관,0,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,22,0,3,0.00,21.15,19,15,4,0,3.00,17.76,125,50,3,10.0,78.62,0,0,0,0.00,0.00,9,1,5,3,0,0.2,11.69,1,0,6,S,0.0125,1
7,현대건설,수원체육관,0,2020-10-17,1,김연견,수원체육관,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0.00,0.00,10,8,2,0,1.60,9.35,4,1,0,0.2,2.52,0,0,0,0.00,0.00,0,0,0,0,0,0.0,0.00,0,0,0,Li,0.0000,0
8,현대건설,수원체육관,0,2020-10-17,1,김주하,수원체육관,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0.00,0.00,16,14,2,0,2.80,14.95,4,0,0,0.0,2.52,22,10,3,31.82,24.18,0,0,0,0,0,0.0,0.00,0,0,0,Li,0.0000,0
0,GS칼텍스,수원체육관,0,2020-10-17,0,러츠,수원체육관,0,33,66,26,4,6,39.39,42.31,37,12,2,4,32.43,46.25,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,14,7,0,2,50.00,87.50,0,0,0,0,0.00,0.00,15,7,2,0,46.67,30.00,13,2,3,0.40,12.75,9,5,4,0,1.00,9.57,5,0,1,0.0,3.16,0,0,0,0.00,0.00,15,5,3,2,0,1.0,20.83,2,0,10,R,0.4125,1


<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# 위의 과정을 함수화
- get_result
    - parameter : driver
    - return : set_result, date, date, team1, team2
- clean_result
    - parameter : set_result, date, date, team1, team2
    - return : team

In [13]:
def get_result(driver) :
    #--------------------- table 태그 읽기 - 팀 별 1 ~ 5 세트 결과 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    set_result = pd.read_html(str(table))[1] # 1 ~ 5 세트 별 득점 결과

    #--------------------- 날짜 정보 읽기 ---------------------#
    date_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > thead > tr > th'
    date = soup.select_one(date_selector).get_text()
    date = date.replace('\xa0', '').split('/')[0].strip().split()
    date = pd.to_datetime(date[0][:-1] + '/' + date[1][:-1] + '/' + date[2][:-1])

    #
    # 첫 번째 팀 정보 읽기
    #

    #--------------------- 버튼 클릭 - 선수 기록 ---------------------#
    x_path = '//*[@id="wrp_content"]/article[2]/ul/li[2]/a/span' # 선수 기록 버튼
    cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)

    #--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')

    team1 = pd.read_html(str(table))[3] # 명단 table
    team1_info = pd.read_html(str(table))[4] # 출전 세트, 득점, 공격 종합 table
    team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
    team1 = pd.concat([team1, team1_info], axis = 1)

    #--------------------- 나머지 기록 읽기 ---------------------#
    for _ in range(4) :
        #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
        x_path = '//*[@id="team1"]/div/div[1]/a[2]' # 다음 기록 버튼
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)


        #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')

        team1_info = pd.read_html(str(table))[4] # 다음 기록
        team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
        team1 = pd.concat([team1, team1_info], axis = 1)


    #
    # 두 번째 팀 정보 읽기
    #

    #--------------------- 버튼 클릭 - 두 번째 팀 ---------------------#
    try :
        x_path = '//*[@id="tab2"]/div[3]/ul/li[2]/a/span' # 두 번째 팀 버튼
        cursor = driver.find_element(By.XPATH,x_path)
    except :
        try :
            x_path = '//*[@id="tab2"]/div[2]/ul/li[2]/a/span' # 두 번째 팀 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        except :
            x_path = '//*[@id="tab2"]/div[1]/ul/li[2]/a/span' # 두 번째 팀 버튼
            cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)

    #--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')

    team2 = pd.read_html(str(table))[5] # 명단 table
    team2_info = pd.read_html(str(table))[6] # 출전 세트, 득점, 공격 종합 table
    team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
    team2 = pd.concat([team2, team2_info], axis = 1)

    #--------------------- 나머지 기록 읽기 ---------------------#
    for _ in range(4) :
        #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
        x_path = '//*[@id="team2"]/div/div[1]/a[2]' # 다음 기록 버튼
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)


        #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')

        team2_info = pd.read_html(str(table))[6] # 다음 기록
        team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
        team2 = pd.concat([team2, team2_info], axis = 1)
    
    return set_result, date, team1, team2

In [14]:
def clean_result(set_result, date, team1, team2) :
    # No. 열 삭제
    team1 = team1.drop('No.', axis = 1)
    team2 = team2.drop('No.', axis = 1)

    
    # 3경기 이상 출전한 선수만 유지 (출전 세트 결측치가 3개 이상인 선수 제거)
    subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

    # thresh = 3 : 정상 값(Not NaN)이 3개 이상만 유지
    team1 = team1.dropna(subset = subset, thresh = 3)
    team2 = team2.dropna(subset = subset, thresh = 3)

    # 스타팅멤버 표시 (3세트 이상 스타팅멤버라면 1 아니면 0)
    team1['스타팅멤버'] = 0
    team2['스타팅멤버'] = 0

    team1.loc[(team1.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
    team2.loc[(team2.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
    
    
    # 출전 세트 열 삭제
    subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

    team1 = team1.drop(subset, axis = 1)
    team2 = team2.drop(subset, axis = 1)

    
    # 이름과 포지션 분리하기
    # "이름" 열 : 이름 정보
    # "포지션" 열 : 포지션 정보
    team1['포지션'] = team1['이름'].str.split().str[1].str.strip('()')
    team1['이름'] = team1['이름'].str.split().str[0]

    team2['포지션'] = team2['이름'].str.split().str[1].str.strip('()')
    team2['이름'] = team2['이름'].str.split().str[0]

    
    # 선수 별 득점 점유율 구하기 (개인 별 득점을 팀 전체의 득점으로 나누기)
    team1_total_score = team1['득점_득점'].sum()
    team2_total_score = team2['득점_득점'].sum()
    team1['득점점유율'] = team1['득점_득점'] / team1_total_score
    team2['득점점유율'] = team2['득점_득점'] / team2_total_score
    
    
    # 컬럼 선택
    columns = [
      '이름',
      '득점_득점',
      '공격종합_시도',
      '공격종합_성공',
      '공격종합_공격차단',
      '공격종합_범실',
      '공격종합_성공률',
      '공격종합_점유율',
      '오픈_시도',
      '오픈_성공',
      '오픈_공격차단',
      '오픈_범실',
      '오픈_성공률',
      '오픈_점유율',
      '시간차_시도',
      '시간차_성공',
      '시간차_공격차단',
      '시간차_범실',
      '시간차_성공률',
      '시간차_점유율',
      '이동_시도',
      '이동_성공',
      '이동_공격차단',
      '이동_범실',
      '이동_성공률',
      '이동_점유율',
      '후위_시도',
      '후위_성공',
      '후위_공격차단',
      '후위_범실',
      '후위_성공률',
      '후위_점유율',
      '속공_시도',
      '속공_성공',
      '속공_공격차단',
      '속공_범실',
      '속공_성공률',
      '속공_점유율',
      '퀵오픈_시도',
      '퀵오픈_성공',
      '퀵오픈_공격차단',
      '퀵오픈_범실',
      '퀵오픈_성공률',
      '퀵오픈_점유율',
      '서브_시도',
      '서브_성공',
      '서브_범실',
      '서브_성공률',
      '서브_점유율',
      '디그_시도',
      '디그_성공',
      '디그_실패',
      '디그_범실',
      '디그_세트당',
      '디그_점유율',
      '세트_시도',
      '세트_성공',
      '세트_범실',
      '세트_세트당',
      '세트_점유율',
      '리시브_시도',
      '리시브_정확',
      '리시브_실패',
      '리시브_세트당',
      '리시브_점유율',
      '블로킹_시도',
      '블로킹_성공',
      '블로킹_유효블락',
      '블로킹_실패',
      '블로킹_범실',
      '블로킹_세트당',
      '블로킹_점유율',
      '블로킹_어시스트',
      '벌칙_벌칙',
      '범실_범실',
      '포지션',
      '득점점유율',
      '스타팅멤버',
    ];
    team1 = team1[columns]
    team2 = team2[columns]
    
    
    # 경기날짜, 팀명, 결과 정보 추가
    team1['팀명'] = set_result.iloc[0,0]
    team2['팀명'] = set_result.iloc[1,0]
    team1['결과'] = 1 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 0
    team2['결과'] = 0 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 1
    team1['경기날짜'] = date
    team2['경기날짜'] = date
    team1['경기장'] = stadium_name
    team1['관중수'] = stadium_crowd
    team2['경기장'] = stadium_name
    team2['관중수'] = stadium_crowd

    # 컬럼 순서 변경
    team1 = team1[['팀명','경기장', '관중수', '경기날짜', '결과'] + columns]
    team2 = team2[['팀명', '경기장', '관중수','경기날짜', '결과'] + columns]
    
    
    # 연결
    team = pd.concat([team1, team2])
    team
    
    return team

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# 모든 경기의 정보 가져오기
- 경기 스케쥴 url 확인
- 여자 배구 상세결과 url 확인
- 각 url 별 함수 적용
- 파일로 저장

#### 경기 스케쥴 url 확인
- 도드람 2017-2018 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2018-03&r_round=
    
- 도드람 2018-2019 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2018-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2019-03&r_round=
    
- 도드람 2018-2020 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=016&team=&yymm=2019-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=016&team=&yymm=2020-03&r_round=

- 도드람 2020-2021 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=017&team=&yymm=2020-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=017&team=&yymm=2021-04&r_round=

- 도드람 2021-2022 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=018&team=&yymm=2021-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=018&team=&yymm=2022-02&r_round=

In [15]:
# 반복문으로 각 url 출력
seasons = [f'0{num}' for num in range(14, 19)]
yymms = [
pd.date_range('2017-10', '2018-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2018-10', '2019-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2019-10', '2020-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2020-10', '2021-05', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2021-10', '2022-03', freq = 'M').strftime('%Y-%m').to_list()
]
# print(seasons)
# print(yymm)

for season, dates in zip(seasons, yymms) :
    for date in dates :
        url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
        print(url)

https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-10&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-11&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-12&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2018-01&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2018-02&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2018-03&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2018-10&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2018-11&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2018-12&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2019-01&r_round=


<br/>
<br/>
<br/>
<br/>

#### 여자 배구 상세결과 url 확인
- 남자 / 여자 XPath 확인
- 상세결과 버튼 XPath 확인
- 경기가 없는 날의 XPath 확인

In [16]:
# url - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-10&r_round=
'''
//*[@id="type1"]/div/table/tbody/tr[1]/td[3] # 남자
//*[@id="type1"]/div/table/tbody/tr[1]/td[10]/a[2] # 상세결과


//*[@id="type1"]/div/table/tbody/tr[2]/td[3] # 여자
//*[@id="type1"]/div/table/tbody/tr[2]/td[10]/a[2] # 상세결과


//*[@id="type1"]/div/table/tbody/tr[3]/td[3] # 남자
//*[@id="type1"]/div/table/tbody/tr[3]/td[10]/a[2] # 상세결과


//*[@id="type1"]/div/table/tbody/tr[4]/td[3] # 여자
//*[@id="type1"]/div/table/tbody/tr[4]/td[10]/a[2] # 상세결과


//*[@id="type1"]/div/table/tbody/tr[5]/td[3] # 경기가 없습니다.


//*[@id="type1"]/div/table/tbody/tr[6]/td[3] # 여자
//*[@id="type1"]/div/table/tbody/tr[6]/td[10]/a[2] # 상세결과
...


'''

'\n//*[@id="type1"]/div/table/tbody/tr[1]/td[3] # 남자\n//*[@id="type1"]/div/table/tbody/tr[1]/td[10]/a[2] # 상세결과\n\n\n//*[@id="type1"]/div/table/tbody/tr[2]/td[3] # 여자\n//*[@id="type1"]/div/table/tbody/tr[2]/td[10]/a[2] # 상세결과\n\n\n//*[@id="type1"]/div/table/tbody/tr[3]/td[3] # 남자\n//*[@id="type1"]/div/table/tbody/tr[3]/td[10]/a[2] # 상세결과\n\n\n//*[@id="type1"]/div/table/tbody/tr[4]/td[3] # 여자\n//*[@id="type1"]/div/table/tbody/tr[4]/td[10]/a[2] # 상세결과\n\n\n//*[@id="type1"]/div/table/tbody/tr[5]/td[3] # 경기가 없습니다.\n\n\n//*[@id="type1"]/div/table/tbody/tr[6]/td[3] # 여자\n//*[@id="type1"]/div/table/tbody/tr[6]/td[10]/a[2] # 상세결과\n...\n\n\n'

In [17]:
## 여자 경기의 상세결과 접근하기

# 경기 스케줄 url 접속
driver = webdriver.Chrome()
url = "https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-10&r_round="
driver.get(url)
time.sleep(2)


# 경기 스케줄을 데이터프레임으로 저장 후 여자 경기의 인덱스를 idx에 저장
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
idx = df[df['구분'] == '여자'].index.to_list()
idx = [i + 1 for i in idx]

dfs = []

# 각 경기의 상세결과 버튼 클릭
for i in idx :
    # 상세결과 버튼 클릭
    try :
        x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
        cursor = driver.find_element(By.XPATH,x_path)
    except :
        x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
        cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)
    
    # 함수 적용
    set_result, date, team1, team2 = get_result(driver)
    team = clean_result(set_result, date, team1, team2)
    dfs.append(team)
    
    # 다시 경기 스케줄 url로 이동
    driver.get(url)
    time.sleep(1)
df = pd.concat(dfs)
df

,팀명,경기장,관중수,경기날짜,결과,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,IBK기업은행,수원체육관,0,2017-10-14,0,메디,24,67,20,4,2,29.85,37.43,41,11,3,1,26.83,46.59,3,3,0,0,100.00,17.65,0,0,0,0,0.00,0.00,12,1,1,0,8.33,57.14,0,0,0,0,0.00,0.00,11,5,0,1,45.45,28.21,17,1,3,0.20,17.89,24,20,4,0,4.00,20.17,7,0,1,0.00,3.72,28,11,2,1.80,28.28,20,3,5,6,1,0.60,21.28,1,0,7,L,0.347826,1
1,IBK기업은행,수원체육관,0,2017-10-14,0,고예림,15,40,12,6,4,30.00,22.35,25,3,5,4,12.00,28.41,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,15,9,1,0,60.00,38.46,18,3,2,0.60,18.95,22,17,5,0,3.40,18.49,7,1,0,0.20,3.72,25,15,1,2.80,25.25,10,0,7,3,0,0.00,10.64,0,0,6,L,0.217391,1
2,IBK기업은행,수원체육관,0,2017-10-14,0,김희진,15,36,10,2,2,27.78,20.11,10,2,0,1,20.00,11.36,9,4,1,1,44.44,52.94,0,0,0,0,0.00,0.00,9,0,1,0,0.00,42.86,0,0,0,0,0.00,0.00,8,4,0,0,50.00,20.51,13,1,3,0.20,13.68,12,11,1,0,2.20,10.08,7,0,1,0.00,3.72,0,0,0,0.00,0.00,14,4,6,3,0,0.80,14.89,1,0,6,R,0.217391,1
3,IBK기업은행,수원체육관,0,2017-10-14,0,김미연,7,16,5,1,0,31.25,8.94,6,1,0,0,16.67,6.82,1,0,1,0,0.00,5.88,3,1,0,0,33.33,42.86,0,0,0,0,0.00,0.00,1,0,0,0,0.00,14.29,5,3,0,0,60.00,12.82,14,0,1,0.00,14.74,6,5,1,0,1.00,5.04,3,0,0,0.00,1.60,1,0,0,0.00,1.01,12,2,2,7,0,0.40,12.77,3,0,1,L,0.101449,1
4,IBK기업은행,수원체육관,0,2017-10-14,0,김수지,6,16,4,1,2,25.00,8.94,2,0,0,0,0.00,2.27,4,2,0,1,50.00,23.53,4,1,1,0,25.00,57.14,0,0,0,0,0.00,0.00,6,1,0,1,16.67,85.71,0,0,0,0,0.00,0.00,14,1,1,0.20,14.74,11,10,1,0,2.00,9.24,7,0,0,0.00,3.72,0,0,0,0.00,0.00,28,1,13,9,0,0.20,29.79,2,0,3,C,0.086957,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,IBK기업은행,수원체육관,0,2017-10-31,1,김미연,2,7,2,1,0,28.57,6.80,4,1,0,0,25.00,7.02,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,1,1,0,0,100.00,33.33,2,0,1,0,0.00,18.18,2,0,0,0.00,2.70,2,2,0,0,0.67,2.60,0,0,0,0.00,0.00,2,0,0,0.00,3.77,7,0,4,3,0,0.00,14.58,0,0,0,L,0.036364,0
5,IBK기업은행,수원체육관,0,2017-10-31,1,김현지,2,2,1,0,0,50.00,1.94,1,0,0,0,0.00,1.75,0,0,0,0,0.00,0.00,1,1,0,0,100.00,16.67,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,8,0,0,0.00,10.81,7,5,2,0,1.67,9.09,3,0,0,0.00,2.83,1,1,0,0.33,1.89,8,1,4,3,0,0.33,16.67,0,0,0,C,0.036364,0
6,IBK기업은행,수원체육관,0,2017-10-31,1,염혜선,1,2,1,0,0,50.00,1.94,2,1,0,0,50.00,3.51,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,12,0,1,0.00,16.22,13,12,1,0,4.00,16.88,60,28,0,9.33,56.60,0,0,0,0.00,0.00,3,0,0,1,0,0.00,6.25,1,0,1,S,0.018182,1
7,IBK기업은행,수원체육관,0,2017-10-31,1,노란,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,9,1,1,0.33,12.16,1,1,0,0,0.33,1.30,1,0,0,0.00,0.94,0,0,0,0.00,0.00,0,0,0,0,0,0.00,0.00,0,0,1,L,0.018182,0


<br/>
<br/>
<br/>
<br/>

#### 각 url 별 함수 적용

In [18]:
dfs = []
driver = webdriver.Chrome()

for season, dates in zip(seasons, yymms) :
    for date in dates :
        # 경기 스케줄 url 접속
        url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
        driver.get(url)
        time.sleep(1)


        # 경기 스케줄을 데이터프레임으로 저장 후 여자 경기의 인덱스를 idx에 저장
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')
        try :
            df = pd.read_html(str(table))[0]
            idx = df[df['구분'] == '여자'].index.to_list()
        except :
            df = pd.read_html(str(table))[2]
            idx = df[df['구분'] == '여자'].index.to_list()
        idx = [i + 1 for i in idx]

        

        # 각 경기의 상세결과 버튼 클릭
        for i in idx :
            # 상세결과 버튼 클릭
            try :
                x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            except :
                x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            cursor.click()
            time.sleep(1)

            # 함수 적용
            set_result, date, team1, team2 = get_result(driver)
            team = clean_result(set_result, date, team1, team2)
            dfs.append(team)

            # 다시 경기 스케줄 url로 이동
            driver.get(url)
            time.sleep(1)
df = pd.concat(dfs)
df

,팀명,경기장,관중수,경기날짜,결과,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,IBK기업은행,수원체육관,0,2017-10-14,0,메디,24,67,20,4,2,29.85,37.43,41,11,3,1,26.83,46.59,3,3,0,0,100.00,17.65,0,0,0,0,0.00,0.00,12,1,1,0,8.33,57.14,0,0,0,0,0.00,0.00,11,5,0,1,45.45,28.21,17,1,3,0.20,17.89,24,20,4,0,4.00,20.17,7,0,1,0.00,3.72,28,11,2,1.80,28.28,20,3,5,6,1,0.60,21.28,1,0,7,L,0.347826,1
1,IBK기업은행,수원체육관,0,2017-10-14,0,고예림,15,40,12,6,4,30.00,22.35,25,3,5,4,12.00,28.41,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,15,9,1,0,60.00,38.46,18,3,2,0.60,18.95,22,17,5,0,3.40,18.49,7,1,0,0.20,3.72,25,15,1,2.80,25.25,10,0,7,3,0,0.00,10.64,0,0,6,L,0.217391,1
2,IBK기업은행,수원체육관,0,2017-10-14,0,김희진,15,36,10,2,2,27.78,20.11,10,2,0,1,20.00,11.36,9,4,1,1,44.44,52.94,0,0,0,0,0.00,0.00,9,0,1,0,0.00,42.86,0,0,0,0,0.00,0.00,8,4,0,0,50.00,20.51,13,1,3,0.20,13.68,12,11,1,0,2.20,10.08,7,0,1,0.00,3.72,0,0,0,0.00,0.00,14,4,6,3,0,0.80,14.89,1,0,6,R,0.217391,1
3,IBK기업은행,수원체육관,0,2017-10-14,0,김미연,7,16,5,1,0,31.25,8.94,6,1,0,0,16.67,6.82,1,0,1,0,0.00,5.88,3,1,0,0,33.33,42.86,0,0,0,0,0.00,0.00,1,0,0,0,0.00,14.29,5,3,0,0,60.00,12.82,14,0,1,0.00,14.74,6,5,1,0,1.00,5.04,3,0,0,0.00,1.60,1,0,0,0.00,1.01,12,2,2,7,0,0.40,12.77,3,0,1,L,0.101449,1
4,IBK기업은행,수원체육관,0,2017-10-14,0,김수지,6,16,4,1,2,25.00,8.94,2,0,0,0,0.00,2.27,4,2,0,1,50.00,23.53,4,1,1,0,25.00,57.14,0,0,0,0,0.00,0.00,6,1,0,1,16.67,85.71,0,0,0,0,0.00,0.00,14,1,1,0.20,14.74,11,10,1,0,2.00,9.24,7,0,0,0.00,3.72,0,0,0,0.00,0.00,28,1,13,9,0,0.20,29.79,2,0,3,C,0.086957,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,KGC인삼공사,수원체육관,0,2022-02-28,0,한송이,4,7,2,2,0,28.57,7.00,2,0,1,0,0.00,4.76,0,0,0,0,0.00,0.00,2,0,1,0,0.00,100.00,0,0,0,0,0.00,0.00,3,2,0,0,66.67,50.00,0,0,0,0,0.00,0.00,8,0,0,0.00,15.09,7,6,1,0,2.00,11.67,0,0,0,0.00,0.00,0,0,0,0.00,0.00,14,2,5,5,0,0.67,31.11,2,0,0,C,0.088889,1
4,KGC인삼공사,수원체육관,0,2022-02-28,0,정호영,3,5,3,1,0,60.00,5.00,3,2,1,0,66.67,7.14,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,2,1,0,0,50.00,33.33,0,0,0,0,0.00,0.00,4,0,0,0.00,7.55,1,1,0,0,0.33,1.67,0,0,0,0.00,0.00,0,0,0,0.00,0.00,2,0,0,2,0,0.00,4.44,3,0,0,C,0.066667,1
5,KGC인삼공사,수원체육관,0,2022-02-28,0,고의정,1,3,0,0,0,0.00,3.00,2,0,0,0,0.00,4.76,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,1,0,0,0,0.00,3.23,4,1,1,0.33,7.55,3,2,1,0,0.67,5.00,0,0,0,0.00,0.00,3,0,0,0.00,4.23,0,0,0,0,0,0.00,0.00,0,0,1,L,0.022222,0
6,KGC인삼공사,수원체육관,0,2022-02-28,0,하효림,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,9,1,0,0.33,16.98,3,2,1,0,0.67,5.00,54,22,0,7.33,51.92,1,0,0,0.00,1.41,4,0,2,1,0,0.00,8.89,2,0,0,S,0.022222,0


- 수동 실행 (시즌 별)

In [19]:
dfs = []
season = seasons[4]
dates = yymms[4]
driver = webdriver.Chrome()

for date in dates :
    # 경기 스케줄 url 접속
    url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
    driver.get(url)
    time.sleep(1)


    # 경기 스케줄을 데이터프레임으로 저장 후 여자 경기의 인덱스를 idx에 저장
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    try :
        df = pd.read_html(str(table))[0]
        idx = df[df['구분'] == '여자'].index.to_list()
    except :
        df = pd.read_html(str(table))[2]
        idx = df[df['구분'] == '여자'].index.to_list()
    idx = [i + 1 for i in idx]



    # 각 경기의 상세결과 버튼 클릭
    for i in idx :
        # 상세결과 버튼 클릭
        try :
            x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        except :
            x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)

        # 함수 적용
        set_result, date, team1, team2 = get_result(driver)
        team = clean_result(set_result, date, team1, team2)
        dfs.append(team)

        # 다시 경기 스케줄 url로 이동
        driver.get(url)
        time.sleep(1)
        
# csv 파일로 저장
df = pd.concat(dfs)
df.to_csv(f'volleyball_{season}.csv', index = False)
df = pd.read_csv(f'volleyball_{season}.csv')
df

,팀명,경기장,관중수,경기날짜,결과,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,GS칼텍스,수원체육관,0,2021-10-16,1,모마,20,45,19,7,2,42.22,37.19,26,13,3,1,50.00,41.94,4,2,1,0,50.0,50.0,0,0,0,0,0.0,0.0,13,4,3,0,30.77,100.0,0,0,0,0,0.00,0.00,2,0,0,1,0.00,7.14,8,1,1,0.33,10.53,12,10,2,0,3.33,16.22,4,1,0,0.33,3.25,0,0,0,0.00,0.00,7,0,4,2,0,0.00,13.73,3,0,3,R,0.350877,1
1,GS칼텍스,수원체육관,0,2021-10-16,1,강소휘,13,25,11,0,2,44.00,20.66,15,7,0,0,46.67,24.19,3,0,0,1,0.0,37.5,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,7,4,0,1,57.14,25.00,12,1,1,0.33,15.79,13,11,2,0,3.67,17.57,2,0,0,0.00,1.63,31,16,2,45.16,46.97,6,1,0,4,0,0.33,11.76,0,0,3,L,0.228070,1
2,GS칼텍스,수원체육관,0,2021-10-16,1,유서연,12,27,12,2,1,44.44,22.31,13,6,1,1,46.15,20.97,1,0,1,0,0.0,12.5,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,13,6,0,0,46.15,46.43,12,0,3,0.00,15.79,15,14,1,0,4.67,20.27,2,0,0,0.00,1.63,13,7,1,46.15,19.70,7,0,5,1,0,0.00,13.73,0,0,4,L,0.210526,1
3,GS칼텍스,수원체육관,0,2021-10-16,1,한수지,8,5,4,0,0,80.00,4.13,1,1,0,0,100.00,1.61,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,4,3,0,0,75.00,44.44,0,0,0,0,0.00,0.00,12,1,1,0.33,15.79,5,4,1,0,1.33,6.76,0,0,0,0.00,0.00,0,0,0,0.00,0.00,14,3,5,3,0,1.00,27.45,0,0,1,C,0.140351,1
4,GS칼텍스,수원체육관,0,2021-10-16,1,김유리,2,5,2,0,0,40.00,4.13,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,1,0,0,0,0.0,100.0,0,0,0,0,0.00,0.0,4,2,0,0,50.00,44.44,0,0,0,0,0.00,0.00,8,0,2,0.00,10.53,0,0,0,0,0.00,0.00,0,0,0,0.00,0.00,0,0,0,0.00,0.00,6,0,3,2,0,0.00,11.76,1,0,2,C,0.035088,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,KGC인삼공사,수원체육관,0,2022-02-28,0,한송이,4,7,2,2,0,28.57,7.00,2,0,1,0,0.00,4.76,0,0,0,0,0.0,0.0,2,0,1,0,0.0,100.0,0,0,0,0,0.00,0.0,3,2,0,0,66.67,50.00,0,0,0,0,0.00,0.00,8,0,0,0.00,15.09,7,6,1,0,2.00,11.67,0,0,0,0.00,0.00,0,0,0,0.00,0.00,14,2,5,5,0,0.67,31.11,2,0,0,C,0.088889,1
1727,KGC인삼공사,수원체육관,0,2022-02-28,0,정호영,3,5,3,1,0,60.00,5.00,3,2,1,0,66.67,7.14,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,2,1,0,0,50.00,33.33,0,0,0,0,0.00,0.00,4,0,0,0.00,7.55,1,1,0,0,0.33,1.67,0,0,0,0.00,0.00,0,0,0,0.00,0.00,2,0,0,2,0,0.00,4.44,3,0,0,C,0.066667,1
1728,KGC인삼공사,수원체육관,0,2022-02-28,0,고의정,1,3,0,0,0,0.00,3.00,2,0,0,0,0.00,4.76,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,1,0,0,0,0.00,3.23,4,1,1,0.33,7.55,3,2,1,0,0.67,5.00,0,0,0,0.00,0.00,3,0,0,0.00,4.23,0,0,0,0,0,0.00,0.00,0,0,1,L,0.022222,0
1729,KGC인삼공사,수원체육관,0,2022-02-28,0,하효림,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,9,1,0,0.33,16.98,3,2,1,0,0.67,5.00,54,22,0,7.33,51.92,1,0,0,0.00,1.41,4,0,2,1,0,0.00,8.89,2,0,0,S,0.022222,0


<br/>
<br/>
<br/>
<br/>

#### 데이터 연결

In [21]:
dfs = []

dfs.append(pd.read_csv(f'volleyball_018.csv'))
df = pd.concat(dfs)
df.to_csv('volleyball.csv', index = False)
df = pd.read_csv('volleyball.csv')
df

,팀명,경기장,관중수,경기날짜,결과,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,GS칼텍스,수원체육관,0,2021-10-16,1,모마,20,45,19,7,2,42.22,37.19,26,13,3,1,50.00,41.94,4,2,1,0,50.0,50.0,0,0,0,0,0.0,0.0,13,4,3,0,30.77,100.0,0,0,0,0,0.00,0.00,2,0,0,1,0.00,7.14,8,1,1,0.33,10.53,12,10,2,0,3.33,16.22,4,1,0,0.33,3.25,0,0,0,0.00,0.00,7,0,4,2,0,0.00,13.73,3,0,3,R,0.350877,1
1,GS칼텍스,수원체육관,0,2021-10-16,1,강소휘,13,25,11,0,2,44.00,20.66,15,7,0,0,46.67,24.19,3,0,0,1,0.0,37.5,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,7,4,0,1,57.14,25.00,12,1,1,0.33,15.79,13,11,2,0,3.67,17.57,2,0,0,0.00,1.63,31,16,2,45.16,46.97,6,1,0,4,0,0.33,11.76,0,0,3,L,0.228070,1
2,GS칼텍스,수원체육관,0,2021-10-16,1,유서연,12,27,12,2,1,44.44,22.31,13,6,1,1,46.15,20.97,1,0,1,0,0.0,12.5,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,13,6,0,0,46.15,46.43,12,0,3,0.00,15.79,15,14,1,0,4.67,20.27,2,0,0,0.00,1.63,13,7,1,46.15,19.70,7,0,5,1,0,0.00,13.73,0,0,4,L,0.210526,1
3,GS칼텍스,수원체육관,0,2021-10-16,1,한수지,8,5,4,0,0,80.00,4.13,1,1,0,0,100.00,1.61,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,4,3,0,0,75.00,44.44,0,0,0,0,0.00,0.00,12,1,1,0.33,15.79,5,4,1,0,1.33,6.76,0,0,0,0.00,0.00,0,0,0,0.00,0.00,14,3,5,3,0,1.00,27.45,0,0,1,C,0.140351,1
4,GS칼텍스,수원체육관,0,2021-10-16,1,김유리,2,5,2,0,0,40.00,4.13,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,1,0,0,0,0.0,100.0,0,0,0,0,0.00,0.0,4,2,0,0,50.00,44.44,0,0,0,0,0.00,0.00,8,0,2,0.00,10.53,0,0,0,0,0.00,0.00,0,0,0,0.00,0.00,0,0,0,0.00,0.00,6,0,3,2,0,0.00,11.76,1,0,2,C,0.035088,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,KGC인삼공사,수원체육관,0,2022-02-28,0,한송이,4,7,2,2,0,28.57,7.00,2,0,1,0,0.00,4.76,0,0,0,0,0.0,0.0,2,0,1,0,0.0,100.0,0,0,0,0,0.00,0.0,3,2,0,0,66.67,50.00,0,0,0,0,0.00,0.00,8,0,0,0.00,15.09,7,6,1,0,2.00,11.67,0,0,0,0.00,0.00,0,0,0,0.00,0.00,14,2,5,5,0,0.67,31.11,2,0,0,C,0.088889,1
1727,KGC인삼공사,수원체육관,0,2022-02-28,0,정호영,3,5,3,1,0,60.00,5.00,3,2,1,0,66.67,7.14,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,2,1,0,0,50.00,33.33,0,0,0,0,0.00,0.00,4,0,0,0.00,7.55,1,1,0,0,0.33,1.67,0,0,0,0.00,0.00,0,0,0,0.00,0.00,2,0,0,2,0,0.00,4.44,3,0,0,C,0.066667,1
1728,KGC인삼공사,수원체육관,0,2022-02-28,0,고의정,1,3,0,0,0,0.00,3.00,2,0,0,0,0.00,4.76,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,1,0,0,0,0.00,3.23,4,1,1,0.33,7.55,3,2,1,0,0.67,5.00,0,0,0,0.00,0.00,3,0,0,0.00,4.23,0,0,0,0,0,0.00,0.00,0,0,1,L,0.022222,0
1729,KGC인삼공사,수원체육관,0,2022-02-28,0,하효림,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,9,1,0,0.33,16.98,3,2,1,0,0.67,5.00,54,22,0,7.33,51.92,1,0,0,0.00,1.41,4,0,2,1,0,0.00,8.89,2,0,0,S,0.022222,0


In [22]:
# 경기날짜의 데이터타입 변경 (datetime64)
df['경기날짜'] = df['경기날짜'].astype('datetime64')
df.dtypes

팀명               object
경기장              object
관중수               int64
경기날짜     datetime64[ns]
결과                int64
              ...      
벌칙_벌칙             int64
범실_범실             int64
포지션              object
득점점유율           float64
스타팅멤버             int64
Length: 83, dtype: object

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# 머신러닝
- 데이터 변경
- 데이터 분리
- 모델링

#### 데이터 변경
- A, B팀의 스타팅 멤버들의 직전 경기력으로 승패 예측

In [23]:
# 한 경기당 스타팅멤버의 수 확인
df.groupby(['경기날짜', '팀명'])['스타팅멤버'].sum().value_counts()

6    121
5     55
4     18
3     10
7      3
2      3
Name: 스타팅멤버, dtype: int64

In [24]:
# 스타팅멤버만 선택
df_starting = df[df['스타팅멤버'] == 1]
df_starting

,팀명,경기장,관중수,경기날짜,결과,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,GS칼텍스,수원체육관,0,2021-10-16,1,모마,20,45,19,7,2,42.22,37.19,26,13,3,1,50.00,41.94,4,2,1,0,50.0,50.0,0,0,0,0,0.0,0.0,13,4,3,0,30.77,100.0,0,0,0,0,0.00,0.00,2,0,0,1,0.00,7.14,8,1,1,0.33,10.53,12,10,2,0,3.33,16.22,4,1,0,0.33,3.25,0,0,0,0.00,0.00,7,0,4,2,0,0.00,13.73,3,0,3,R,0.350877,1
1,GS칼텍스,수원체육관,0,2021-10-16,1,강소휘,13,25,11,0,2,44.00,20.66,15,7,0,0,46.67,24.19,3,0,0,1,0.0,37.5,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,7,4,0,1,57.14,25.00,12,1,1,0.33,15.79,13,11,2,0,3.67,17.57,2,0,0,0.00,1.63,31,16,2,45.16,46.97,6,1,0,4,0,0.33,11.76,0,0,3,L,0.228070,1
2,GS칼텍스,수원체육관,0,2021-10-16,1,유서연,12,27,12,2,1,44.44,22.31,13,6,1,1,46.15,20.97,1,0,1,0,0.0,12.5,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,13,6,0,0,46.15,46.43,12,0,3,0.00,15.79,15,14,1,0,4.67,20.27,2,0,0,0.00,1.63,13,7,1,46.15,19.70,7,0,5,1,0,0.00,13.73,0,0,4,L,0.210526,1
3,GS칼텍스,수원체육관,0,2021-10-16,1,한수지,8,5,4,0,0,80.00,4.13,1,1,0,0,100.00,1.61,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,4,3,0,0,75.00,44.44,0,0,0,0,0.00,0.00,12,1,1,0.33,15.79,5,4,1,0,1.33,6.76,0,0,0,0.00,0.00,0,0,0,0.00,0.00,14,3,5,3,0,1.00,27.45,0,0,1,C,0.140351,1
5,GS칼텍스,수원체육관,0,2021-10-16,1,안혜진,2,1,0,0,0,0.00,0.83,1,0,0,0,0.00,1.61,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,21,2,1,0.67,27.63,7,6,1,0,2.00,9.46,100,46,1,15.33,81.30,0,0,0,0.00,0.00,6,0,3,1,0,0.00,11.76,0,0,2,S,0.035088,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,KGC인삼공사,수원체육관,0,2022-02-28,0,옐레나,21,41,18,1,1,43.90,41.00,17,6,1,0,35.29,40.48,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,18,9,0,1,50.00,100.0,0,0,0,0,0.00,0.00,6,3,0,0,50.00,19.35,7,0,0,0.00,13.21,14,9,4,1,3.00,23.33,1,0,0,0.00,0.96,1,0,0,0.00,1.41,11,3,3,5,0,1.00,24.44,0,0,2,R,0.466667,1
1724,KGC인삼공사,수원체육관,0,2022-02-28,0,이소영,9,21,6,3,1,28.57,21.00,8,0,3,0,0.00,19.05,1,0,0,1,0.0,100.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,12,6,0,0,50.00,38.71,6,0,1,0.00,11.32,12,10,2,0,3.33,20.00,7,1,0,0.33,6.73,24,10,2,33.33,33.80,8,3,1,1,0,1.00,17.78,0,0,2,L,0.200000,1
1725,KGC인삼공사,수원체육관,0,2022-02-28,0,박혜민,6,20,6,3,2,30.00,20.00,8,2,2,0,25.00,19.05,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,12,4,1,2,33.33,38.71,7,0,2,0.00,13.21,8,7,1,0,2.33,13.33,2,0,0,0.00,1.92,13,3,1,15.38,18.31,3,0,1,1,0,0.00,6.67,0,0,4,L,0.133333,1
1726,KGC인삼공사,수원체육관,0,2022-02-28,0,한송이,4,7,2,2,0,28.57,7.00,2,0,1,0,0.00,4.76,0,0,0,0,0.0,0.0,2,0,1,0,0.0,100.0,0,0,0,0,0.00,0.0,3,2,0,0,66.67,50.00,0,0,0,0,0.00,0.00,8,0,0,0.00,15.09,7,6,1,0,2.00,11.67,0,0,0,0.00,0.00,0,0,0,0.00,0.00,14,2,5,5,0,0.67,31.11,2,0,0,C,0.088889,1


In [25]:
## 경기 당 스타팅멤버가 5명 이상만 선택

# 스타팅멤버가 5명 미만이 포함된 날짜 선택
temp = (df_starting.groupby(['경기날짜', '팀명'])[['스타팅멤버']].sum() < 5).reset_index()
drop_dates = temp.loc[temp['스타팅멤버'], '경기날짜'].unique()

# drop_dates를 제외한 날짜만 선택
df_starting = df_starting[~df_starting['경기날짜'].isin(drop_dates)]

# 스타팅멤버 다시 확인
df_starting.groupby(['경기날짜', '팀명'])['스타팅멤버'].sum().value_counts()

6    105
5     42
7      3
Name: 스타팅멤버, dtype: int64

In [26]:
df_starting.head()

,팀명,경기장,관중수,경기날짜,결과,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,GS칼텍스,수원체육관,0,2021-10-16,1,모마,20,45,19,7,2,42.22,37.19,26,13,3,1,50.00,41.94,4,2,1,0,50.0,50.0,0,0,0,0,0.0,0.0,13,4,3,0,30.77,100.0,0,0,0,0,0.0,0.00,2,0,0,1,0.00,7.14,8,1,1,0.33,10.53,12,10,2,0,3.33,16.22,4,1,0,0.33,3.25,0,0,0,0.00,0.00,7,0,4,2,0,0.00,13.73,3,0,3,R,0.350877,1
1,GS칼텍스,수원체육관,0,2021-10-16,1,강소휘,13,25,11,0,2,44.00,20.66,15,7,0,0,46.67,24.19,3,0,0,1,0.0,37.5,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.00,7,4,0,1,57.14,25.00,12,1,1,0.33,15.79,13,11,2,0,3.67,17.57,2,0,0,0.00,1.63,31,16,2,45.16,46.97,6,1,0,4,0,0.33,11.76,0,0,3,L,0.228070,1
2,GS칼텍스,수원체육관,0,2021-10-16,1,유서연,12,27,12,2,1,44.44,22.31,13,6,1,1,46.15,20.97,1,0,1,0,0.0,12.5,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.00,13,6,0,0,46.15,46.43,12,0,3,0.00,15.79,15,14,1,0,4.67,20.27,2,0,0,0.00,1.63,13,7,1,46.15,19.70,7,0,5,1,0,0.00,13.73,0,0,4,L,0.210526,1
3,GS칼텍스,수원체육관,0,2021-10-16,1,한수지,8,5,4,0,0,80.00,4.13,1,1,0,0,100.00,1.61,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,4,3,0,0,75.0,44.44,0,0,0,0,0.00,0.00,12,1,1,0.33,15.79,5,4,1,0,1.33,6.76,0,0,0,0.00,0.00,0,0,0,0.00,0.00,14,3,5,3,0,1.00,27.45,0,0,1,C,0.140351,1
5,GS칼텍스,수원체육관,0,2021-10-16,1,안혜진,2,1,0,0,0,0.00,0.83,1,0,0,0,0.00,1.61,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.00,0,0,0,0,0.00,0.00,21,2,1,0.67,27.63,7,6,1,0,2.00,9.46,100,46,1,15.33,81.30,0,0,0,0.00,0.00,6,0,3,1,0,0.00,11.76,0,0,2,S,0.035088,1


In [27]:
# 데이터프레임 변경 (날짜, 팀 별 출전 선수 명단과 결과)
df_starting5 = df_starting.groupby(['경기날짜', '팀명'])[['이름', '결과']].agg({'이름' : lambda data : ' '.join(data).split(), '결과' : any})
df_starting5 = pd.DataFrame(df_starting5).reset_index()
df_starting5['이름'] = df_starting5['이름'].str[:5]
df_starting5.head()

,경기날짜,팀명,이름,결과
0,2021-10-16,GS칼텍스,"[모마, 강소휘, 유서연, 한수지, 안혜진]",True
1,2021-10-16,흥국생명,"[캣벨, 김미연, 이주아, 김채연, 박혜진]",False
2,2021-10-17,IBK기업은행,"[라셈, 김수지, 김주향, 표승주, 김희진]",False
3,2021-10-17,현대건설,"[야스민, 양효진, 이다현, 고예림, 황민경]",True
4,2021-10-19,KGC인삼공사,"[이소영, 옐레나, 한송이, 박혜민, 염혜선]",True


In [28]:
# 경기번호 추가
import numpy as np
total = df_starting5.shape[0] // 2
df_starting5['경기번호'] = np.array([[i,i] for i in range(1, total+1)]).flatten()
df_starting5.head()

,경기날짜,팀명,이름,결과,경기번호
0,2021-10-16,GS칼텍스,"[모마, 강소휘, 유서연, 한수지, 안혜진]",True,1
1,2021-10-16,흥국생명,"[캣벨, 김미연, 이주아, 김채연, 박혜진]",False,1
2,2021-10-17,IBK기업은행,"[라셈, 김수지, 김주향, 표승주, 김희진]",False,2
3,2021-10-17,현대건설,"[야스민, 양효진, 이다현, 고예림, 황민경]",True,2
4,2021-10-19,KGC인삼공사,"[이소영, 옐레나, 한송이, 박혜민, 염혜선]",True,3


In [29]:
# 데이터프레임 변경 (경기 별 두 팀의 이름, 출전 선수 명단과 결과)
def concat_names(data) :
    result = []
    for d in data :
        result += d
    return result
df_game = df_starting5.groupby('경기번호').agg({'경기날짜' : lambda dates : dates.values[0],
                                                '팀명' : lambda teams : ' vs '.join(teams),
                                                '이름' : concat_names,
                                                '결과' : np.argmax}).reset_index()
df_game

,경기번호,경기날짜,팀명,이름,결과
0,1,2021-10-16,GS칼텍스 vs 흥국생명,"[모마, 강소휘, 유서연, 한수지, 안혜진, 캣벨, 김미연, 이주아, 김채연, 박혜진]",0
1,2,2021-10-17,IBK기업은행 vs 현대건설,"[라셈, 김수지, 김주향, 표승주, 김희진, 야스민, 양효진, 이다현, 고예림, 황민경]",1
2,3,2021-10-19,KGC인삼공사 vs 페퍼저축은행,"[이소영, 옐레나, 한송이, 박혜민, 염혜선, 엘리자벳, 하혜진, 박경현, 이한비,...",0
3,4,2021-10-21,IBK기업은행 vs 흥국생명,"[라셈, 김주향, 김수지, 김희진, 조송화, 캣벨, 김미연, 이주아, 김다은, 박혜진]",1
4,5,2021-10-22,GS칼텍스 vs 페퍼저축은행,"[모마, 강소휘, 한수지, 김유리, 유서연, 엘리자벳, 이한비, 박경현, 최가은, ...",0
...,...,...,...,...,...
70,71,2022-02-21,KGC인삼공사 vs 한국도로공사,"[옐레나, 박혜민, 고의정, 박은진, 한송이, 켈시, 박정아, 배유나, 정대영, 문정원]",1
71,72,2022-02-22,IBK기업은행 vs 현대건설,"[김희진, 김주향, 산타나, 김수지, 최정민, 야스민, 양효진, 고예림, 황민경, ...",1
72,73,2022-02-25,KGC인삼공사 vs 현대건설,"[옐레나, 박혜민, 정호영, 이소영, 한송이, 야스민, 양효진, 정지윤, 이다현, ...",0
73,74,2022-02-26,IBK기업은행 vs 흥국생명,"[표승주, 김희진, 산타나, 김수지, 최정민, 캣벨, 김미연, 이주아, 김채연, 박혜진]",0


In [30]:
# 선수의 직전 기록 찾는 함수 정의
def find_previous_record(df, date, name) :
    day1 = pd.Timedelta(days=1)
    date -= day1
    
    while date >= df.loc[0, '경기날짜'] :
        row = df[(df['경기날짜'] == date) & (df['이름'] == name)]
        if len(row) : # 이전 기록 존재
            return row.iloc[0, :]
        date -= day1
    else : # 이전 기록 존재 x
        return []

In [31]:
# 각 선수 별 직전 기록을 새로운 열에 추가
cols = ['공격종합_성공률','오픈_성공률','시간차_성공률','이동_성공률','후위_성공률','속공_성공률','퀵오픈_성공률','서브_성공률','범실_범실']

rows = []
for i in range(len(df_game)) :
    date = df_game.loc[i, '경기날짜']
    names = df_game.loc[i, '이름']
    row = []
    
    for name in names :
        result = find_previous_record(df, date, name)
        if len(result) == 0 :
            row += [np.nan] * len(cols)
        else :
            row += list(result[cols].values.flatten())
    rows.append(row)

    
columns = []
for team in ['A팀', 'B팀'] :
    for player in ['선수1', '선수2', '선수3', '선수4', '선수5'] :
        columns += [f'{team}_{player}_{col}' for col in cols]
        
df_game = pd.concat([df_game, pd.DataFrame(rows, columns = columns)], axis = 1)
df_game.tail()

,경기번호,경기날짜,팀명,이름,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,A팀_선수1_범실_범실,A팀_선수2_공격종합_성공률,A팀_선수2_오픈_성공률,A팀_선수2_시간차_성공률,A팀_선수2_이동_성공률,A팀_선수2_후위_성공률,A팀_선수2_속공_성공률,A팀_선수2_퀵오픈_성공률,A팀_선수2_서브_성공률,A팀_선수2_범실_범실,A팀_선수3_공격종합_성공률,A팀_선수3_오픈_성공률,A팀_선수3_시간차_성공률,A팀_선수3_이동_성공률,A팀_선수3_후위_성공률,A팀_선수3_속공_성공률,A팀_선수3_퀵오픈_성공률,A팀_선수3_서브_성공률,A팀_선수3_범실_범실,A팀_선수4_공격종합_성공률,A팀_선수4_오픈_성공률,A팀_선수4_시간차_성공률,A팀_선수4_이동_성공률,A팀_선수4_후위_성공률,A팀_선수4_속공_성공률,A팀_선수4_퀵오픈_성공률,A팀_선수4_서브_성공률,A팀_선수4_범실_범실,A팀_선수5_공격종합_성공률,A팀_선수5_오픈_성공률,A팀_선수5_시간차_성공률,A팀_선수5_이동_성공률,A팀_선수5_후위_성공률,A팀_선수5_속공_성공률,A팀_선수5_퀵오픈_성공률,A팀_선수5_서브_성공률,A팀_선수5_범실_범실,B팀_선수1_공격종합_성공률,B팀_선수1_오픈_성공률,B팀_선수1_시간차_성공률,B팀_선수1_이동_성공률,B팀_선수1_후위_성공률,B팀_선수1_속공_성공률,B팀_선수1_퀵오픈_성공률,B팀_선수1_서브_성공률,B팀_선수1_범실_범실,B팀_선수2_공격종합_성공률,B팀_선수2_오픈_성공률,B팀_선수2_시간차_성공률,B팀_선수2_이동_성공률,B팀_선수2_후위_성공률,B팀_선수2_속공_성공률,B팀_선수2_퀵오픈_성공률,B팀_선수2_서브_성공률,B팀_선수2_범실_범실,B팀_선수3_공격종합_성공률,B팀_선수3_오픈_성공률,B팀_선수3_시간차_성공률,B팀_선수3_이동_성공률,B팀_선수3_후위_성공률,B팀_선수3_속공_성공률,B팀_선수3_퀵오픈_성공률,B팀_선수3_서브_성공률,B팀_선수3_범실_범실,B팀_선수4_공격종합_성공률,B팀_선수4_오픈_성공률,B팀_선수4_시간차_성공률,B팀_선수4_이동_성공률,B팀_선수4_후위_성공률,B팀_선수4_속공_성공률,B팀_선수4_퀵오픈_성공률,B팀_선수4_서브_성공률,B팀_선수4_범실_범실,B팀_선수5_공격종합_성공률,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실
70,71,2022-02-21,KGC인삼공사 vs 한국도로공사,"[옐레나, 박혜민, 고의정, 박은진, 한송이, 켈시, 박정아, 배유나, 정대영, 문정원]",1,42.11,33.33,100.0,0.0,38.46,0.0,66.67,0.33,1.0,42.86,57.14,0.0,0.0,0.00,0.0,33.33,0.00,3.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,2.0,62.50,50.00,0.0,100.0,0.0,60.00,0.00,0.00,2.0,37.50,0.0,0.0,0.0,0.0,42.86,0.0,0.33,1.0,36.54,30.43,71.43,0.0,30.77,0.0,33.33,0.00,1.0,35.42,42.31,50.00,0.0,50.0,0.00,22.22,0.25,7.0,57.14,57.14,0.0,100.0,0.0,75.0,0.00,0.5,4.0,25.00,28.57,0.00,0.00,0.0,20.00,0.00,0.25,2.0,30.77,0.00,100.0,0.0,0.0,0.00,75.00,0.25,2.0
71,72,2022-02-22,IBK기업은행 vs 현대건설,"[김희진, 김주향, 산타나, 김수지, 최정민, 야스민, 양효진, 고예림, 황민경, ...",1,48.28,50.00,75.0,0.0,60.00,0.0,25.00,0.00,3.0,38.24,11.11,100.0,0.0,0.00,0.0,66.67,0.20,2.0,35.29,10.00,100.0,0.0,0.0,0.00,43.48,0.0,5.0,40.00,0.00,0.0,0.0,0.0,54.55,0.00,0.00,2.0,66.67,0.0,0.0,0.0,0.0,80.00,0.0,0.33,2.0,43.75,33.33,0.00,0.0,63.16,0.0,41.67,0.40,11.0,40.00,40.00,50.00,0.0,0.0,37.50,0.00,0.00,3.0,44.44,50.00,0.0,0.0,0.0,0.0,44.44,0.0,5.0,30.77,22.22,0.00,0.00,0.0,0.00,66.67,0.00,4.0,50.00,100.00,0.0,0.0,0.0,50.00,0.00,0.00,2.0
72,73,2022-02-25,KGC인삼공사 vs 현대건설,"[옐레나, 박혜민, 정호영, 이소영, 한송이, 야스민, 양효진, 정지윤, 이다현, ...",0,35.38,37.84,0.0,0.0,26.09,0.0,60.00,0.25,6.0,31.25,30.43,100.0,0.0,0.00,0.0,29.17,0.25,6.0,75.00,100.00,0.0,0.0,0.0,66.67,0.00,1.0,1.0,40.54,27.78,75.0,0.0,0.0,0.00,36.36,0.00,2.0,50.00,100.0,0.0,0.0,0.0,40.00,0.0,0.00,2.0,33.33,18.75,0.00,0.0,33.33,0.0,80.00,0.00,1.0,23.53,27.27,0.00,0.0,0.0,16.67,0.00,0.00,3.0,28.00,20.00,0.0,0.0,0.0,0.0,66.67,0.0,3.0,28.57,0.00,0.00,33.33,0.0,33.33,0.00,0.00,2.0,55.56,60.00,100.0,0.0,0.0,0.00,33.33,0.25,2.0
73,74,2022-02-26,IBK기업은행 vs 흥국생명,"[표승주, 김희진, 산타나, 김수지, 최정민, 캣벨, 김미연, 이주아, 김채연, 박혜진]",0,26.67,0.00,0.0,0.0,0.00,0.0,50.00,0.00,0.0,32.43,33.33,50.0,0.0,28.57,0.0,33.33,0.25,3.0,39.13,12.50,0.0,0.0,0.0,0.00,57.14,0.0,1.0,33.33,20.00,0.0,50.0,0.0,40.00,0.00,0.25,1.0,42.86,50.0,0.0,0.0,0.0,40.00,0.0,0.00,4.0,34.33,33.33,100.00,0.0,16.67,0.0,38.46,0.25,3.0,37.14,30.00,75.00,0.0,0.0,0.00,35.00,0.25,3.0,16.67,0.00,0.0,0.0,0.0,60.0,0.00,0.0,3.0,38.46,40.00,33.33,0.00,0.0,40.00,0.00,0.00,1.0,20.00,20.00,0.0,0.0,0.0,0.00,0.00,0.25,2.0
74,75,2022-02-28,GS칼텍스 vs KGC인삼공사,"[모마, 유서연, 최은지, 문명화, 안혜진, 옐레나, 이소영, 박혜민, 한송이, 정호영]",0,65.63,75.00,0.0,0.0,75.00,0.0,37.50,0.00,3.0,45.83,10.00,0.0,0.0,0.00,0.0,71.43,0.00,3.0,45.45,28.57,0.0,0.0,0.0,0.00,53.33,0.0,4.0,16.67,0.00,0.0,0.0,0.0,25.00,0.00,0.00,0.0,50.00,50.0,0.0,0.0,0.0,0.00,0.0,0.00,1.0,41.51,37.50,0.00,0.0,48.00,0.0,25.00,0.20,6.0,32.26,27.27,33.33,0.0,0.0,0.00,35.29,0.00,1.0,28.95,15.79,0.0,0.0,0.0,0.0,44.44,0.0,7.0,28.57,50.00,0.00,0.00,0.0,0.00,0.00,0.00,1.0,46.67,33.33,0.0,0.0,0.0,55.56,0.00,0.00,5.0


In [32]:
# '경기번호', '경기날짜', '팀명', '이름' 열 삭제
data = df_game.drop(['경기번호', '경기날짜', '팀명', '이름'], axis = 1)
data

,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,A팀_선수1_범실_범실,A팀_선수2_공격종합_성공률,A팀_선수2_오픈_성공률,A팀_선수2_시간차_성공률,A팀_선수2_이동_성공률,A팀_선수2_후위_성공률,A팀_선수2_속공_성공률,A팀_선수2_퀵오픈_성공률,A팀_선수2_서브_성공률,A팀_선수2_범실_범실,A팀_선수3_공격종합_성공률,A팀_선수3_오픈_성공률,A팀_선수3_시간차_성공률,A팀_선수3_이동_성공률,A팀_선수3_후위_성공률,A팀_선수3_속공_성공률,A팀_선수3_퀵오픈_성공률,A팀_선수3_서브_성공률,A팀_선수3_범실_범실,A팀_선수4_공격종합_성공률,A팀_선수4_오픈_성공률,A팀_선수4_시간차_성공률,A팀_선수4_이동_성공률,A팀_선수4_후위_성공률,A팀_선수4_속공_성공률,A팀_선수4_퀵오픈_성공률,A팀_선수4_서브_성공률,A팀_선수4_범실_범실,A팀_선수5_공격종합_성공률,A팀_선수5_오픈_성공률,A팀_선수5_시간차_성공률,A팀_선수5_이동_성공률,A팀_선수5_후위_성공률,A팀_선수5_속공_성공률,A팀_선수5_퀵오픈_성공률,A팀_선수5_서브_성공률,A팀_선수5_범실_범실,B팀_선수1_공격종합_성공률,B팀_선수1_오픈_성공률,B팀_선수1_시간차_성공률,B팀_선수1_이동_성공률,B팀_선수1_후위_성공률,B팀_선수1_속공_성공률,B팀_선수1_퀵오픈_성공률,B팀_선수1_서브_성공률,B팀_선수1_범실_범실,B팀_선수2_공격종합_성공률,B팀_선수2_오픈_성공률,B팀_선수2_시간차_성공률,B팀_선수2_이동_성공률,B팀_선수2_후위_성공률,B팀_선수2_속공_성공률,B팀_선수2_퀵오픈_성공률,B팀_선수2_서브_성공률,B팀_선수2_범실_범실,B팀_선수3_공격종합_성공률,B팀_선수3_오픈_성공률,B팀_선수3_시간차_성공률,B팀_선수3_이동_성공률,B팀_선수3_후위_성공률,B팀_선수3_속공_성공률,B팀_선수3_퀵오픈_성공률,B팀_선수3_서브_성공률,B팀_선수3_범실_범실,B팀_선수4_공격종합_성공률,B팀_선수4_오픈_성공률,B팀_선수4_시간차_성공률,B팀_선수4_이동_성공률,B팀_선수4_후위_성공률,B팀_선수4_속공_성공률,B팀_선수4_퀵오픈_성공률,B팀_선수4_서브_성공률,B팀_선수4_범실_범실,B팀_선수5_공격종합_성공률,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,27.45,29.41,50.0,0.0,20.00,0.0,0.00,0.25,7.0,36.84,25.00,100.0,0.0,0.00,0.0,50.00,0.00,4.0,43.75,0.00,0.0,42.86,0.0,66.67,0.00,0.25,3.0,36.36,40.00,0.0,50.0,0.0,100.00,0.00,0.00,2.0,25.00,25.00,0.0,0.0,0.0,0.00,0.00,0.00,1.0,35.42,30.00,50.00,0.0,33.33,0.0,57.14,0.67,7.0,38.10,36.36,0.00,0.0,0.0,0.00,57.14,0.33,2.0,50.00,33.33,0.0,33.33,0.0,100.0,0.00,0.0,1.0,35.71,50.00,0.00,0.00,50.0,0.00,25.00,0.00,1.0,50.00,50.00,0.0,0.0,0.0,0.00,0.00,0.00,3.0
4,0,42.22,50.00,50.0,0.0,30.77,0.0,0.00,0.33,3.0,44.00,46.67,0.0,0.0,0.00,0.0,57.14,0.33,3.0,80.00,100.00,0.0,0.00,0.0,75.00,0.00,0.33,1.0,40.00,0.00,0.0,0.0,0.0,50.00,0.00,0.00,2.0,44.44,46.15,0.0,0.0,0.0,0.00,46.15,0.00,4.0,36.36,34.21,0.00,0.0,35.71,0.0,66.67,0.50,7.0,30.00,21.43,0.00,0.0,100.0,0.00,50.00,0.25,2.0,30.00,26.32,50.0,0.00,0.0,0.0,37.50,0.0,8.0,38.46,66.67,100.00,25.00,0.0,20.00,0.00,0.25,2.0,33.33,37.50,50.0,0.0,0.0,0.00,0.00,0.25,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1,42.11,33.33,100.0,0.0,38.46,0.0,66.67,0.33,1.0,42.86,57.14,0.0,0.0,0.00,0.0,33.33,0.00,3.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,2.0,62.50,50.00,0.0,100.0,0.0,60.00,0.00,0.00,2.0,37.50,0.00,0.0,0.0,0.0,42.86,0.00,0.33,1.0,36.54,30.43,71.43,0.0,30.77,0.0,33.33,0.00,1.0,35.42,42.31,50.00,0.0,50.0,0.00,22.22,0.25,7.0,57.14,57.14,0

In [33]:
# 결측치 제거
data = data.dropna()
data

,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,A팀_선수1_범실_범실,A팀_선수2_공격종합_성공률,A팀_선수2_오픈_성공률,A팀_선수2_시간차_성공률,A팀_선수2_이동_성공률,A팀_선수2_후위_성공률,A팀_선수2_속공_성공률,A팀_선수2_퀵오픈_성공률,A팀_선수2_서브_성공률,A팀_선수2_범실_범실,A팀_선수3_공격종합_성공률,A팀_선수3_오픈_성공률,A팀_선수3_시간차_성공률,A팀_선수3_이동_성공률,A팀_선수3_후위_성공률,A팀_선수3_속공_성공률,A팀_선수3_퀵오픈_성공률,A팀_선수3_서브_성공률,A팀_선수3_범실_범실,A팀_선수4_공격종합_성공률,A팀_선수4_오픈_성공률,A팀_선수4_시간차_성공률,A팀_선수4_이동_성공률,A팀_선수4_후위_성공률,A팀_선수4_속공_성공률,A팀_선수4_퀵오픈_성공률,A팀_선수4_서브_성공률,A팀_선수4_범실_범실,A팀_선수5_공격종합_성공률,A팀_선수5_오픈_성공률,A팀_선수5_시간차_성공률,A팀_선수5_이동_성공률,A팀_선수5_후위_성공률,A팀_선수5_속공_성공률,A팀_선수5_퀵오픈_성공률,A팀_선수5_서브_성공률,A팀_선수5_범실_범실,B팀_선수1_공격종합_성공률,B팀_선수1_오픈_성공률,B팀_선수1_시간차_성공률,B팀_선수1_이동_성공률,B팀_선수1_후위_성공률,B팀_선수1_속공_성공률,B팀_선수1_퀵오픈_성공률,B팀_선수1_서브_성공률,B팀_선수1_범실_범실,B팀_선수2_공격종합_성공률,B팀_선수2_오픈_성공률,B팀_선수2_시간차_성공률,B팀_선수2_이동_성공률,B팀_선수2_후위_성공률,B팀_선수2_속공_성공률,B팀_선수2_퀵오픈_성공률,B팀_선수2_서브_성공률,B팀_선수2_범실_범실,B팀_선수3_공격종합_성공률,B팀_선수3_오픈_성공률,B팀_선수3_시간차_성공률,B팀_선수3_이동_성공률,B팀_선수3_후위_성공률,B팀_선수3_속공_성공률,B팀_선수3_퀵오픈_성공률,B팀_선수3_서브_성공률,B팀_선수3_범실_범실,B팀_선수4_공격종합_성공률,B팀_선수4_오픈_성공률,B팀_선수4_시간차_성공률,B팀_선수4_이동_성공률,B팀_선수4_후위_성공률,B팀_선수4_속공_성공률,B팀_선수4_퀵오픈_성공률,B팀_선수4_서브_성공률,B팀_선수4_범실_범실,B팀_선수5_공격종합_성공률,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실
3,1,27.45,29.41,50.0,0.0,20.00,0.0,0.00,0.25,7.0,36.84,25.00,100.0,0.0,0.00,0.00,50.00,0.00,4.0,43.75,0.00,0.0,42.86,0.0,66.67,0.00,0.25,3.0,36.36,40.00,0.0,50.0,0.0,100.00,0.00,0.00,2.0,25.00,25.00,0.0,0.0,0.0,0.00,0.00,0.00,1.0,35.42,30.00,50.00,0.0,33.33,0.0,57.14,0.67,7.0,38.10,36.36,0.00,0.0,0.0,0.00,57.14,0.33,2.0,50.00,33.33,0.0,33.33,0.0,100.0,0.00,0.0,1.0,35.71,50.00,0.00,0.00,50.0,0.00,25.00,0.00,1.0,50.00,50.00,0.0,0.0,0.0,0.00,0.00,0.00,3.0
4,0,42.22,50.00,50.0,0.0,30.77,0.0,0.00,0.33,3.0,44.00,46.67,0.0,0.0,0.00,0.00,57.14,0.33,3.0,80.00,100.00,0.0,0.00,0.0,75.00,0.00,0.33,1.0,40.00,0.00,0.0,0.0,0.0,50.00,0.00,0.00,2.0,44.44,46.15,0.0,0.0,0.0,0.00,46.15,0.00,4.0,36.36,34.21,0.00,0.0,35.71,0.0,66.67,0.50,7.0,30.00,21.43,0.00,0.0,100.0,0.00,50.00,0.25,2.0,30.00,26.32,50.0,0.00,0.0,0.0,37.50,0.0,8.0,38.46,66.67,100.00,25.00,0.0,20.00,0.00,0.25,2.0,33.33,37.50,50.0,0.0,0.0,0.00,0.00,0.25,1.0
6,0,44.74,31.58,0.0,0.0,64.29,0.0,40.00,0.33,5.0,60.00,50.00,0.0,0.0,0.00,83.33,0.00,0.33,0.0,46.15,75.00,0.0,0.00,0.0,37.50,0.00,0.00,1.0,20.00,16.67,0.0,0.0,0.0,0.00,25.00,0.00,1.0,34.78,33.33,75.0,0.0,0.0,0.00,16.67,0.33,1.0,43.82,41.38,0.00,0.0,41.94,0.0,50.00,0.25,5.0,31.25,0.00,0.00,0.0,0.0,55.56,0.00,0.00,2.0,23.53,28.57,0.0,0.00,0.0,0.0,25.00,0.0,1.0,31.25,29.41,0.00,0.00,100.0,0.00,28.57,0.25,1.0,33.33,0.00,0.0,0.0,0.0,100.00,0.00,0.25,2.0
7,1,44.26,37.50,0.0,0.0,44.00,0.0,58.33,0.25,10.0,15.79,7.69,0.0,0.0,0.00,0.00,33.33,0.00,0.0,30.77,25.00,0.0,33.33,0.0,50.00,0.00,0.00,1.0,23.08,40.00,0.0,25.0,0.0,0.00,0.00,0.00,1.0,18.92,19.05,0.0,0.0,0.0,0.00,23.08,0.25,4.0,36.96,30.00,100.00,0.0,41.67,0.0,38.46,0.00,3.0,42.86,36.36,0.00,0.0,0.0,0.00,50.00,0.00,2.0,32.61,25.00,0.0,0.00,50.0,0.0,41.18,0.0,6.0,9.09,25.00,0.00,0.00,0.0,0.00,0.00,0.67,2.0,66.67,100.00,0.0,100.0,0.0,0.00,0.00,0.00,0.0
8,1,61.54,55.00,0.0,0.0,75.00,0.0,100.00,1.00,4.0,42.11,41.67,100.0,0.0,100.00,0.00,20.00,0.67,3.0,66.67,80.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,57.14,0.00,0.0,0.0,0.0,66.67,0.00,0.00,1.0,75.00,100.00,0.0,0.0,0.0,100.00,0.00,0.33,1.0,47.46,46.67,0.00,0.0,38.89,0.0,63.64,1.00,9.0,40.00,36.36,0.00,0.0,0.0,50.00,0.00,0.75,4.0,37.50,0.00,0.0,0.00,0.0,0.0,60.00,0.0,0.0,44.44,0.00,0.00,100.00,0.0,33.33,0.00,0.25,1.0,25.00,10.00,0.0,0.0,100.0,0.00,50.00,0.25,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1,42.11,33.33,100.0,0.0,38.46,0.0,66.67,0.33,1.0,42.86,

<br/>
<br/>
<br/>
<br/>

#### 데이터 분리

In [ ]:
# 결과 열을 제외한 열은 X, 결과 열은 y에 저장
X = data.drop('결과', axis = 1)
y = data['결과']

In [ ]:
# 학습, 검증 데이터 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False)

<br/>
<br/>
<br/>
<br/>

#### 모델링

In [ ]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression()
lg.fit(X_train, y_train)
lg.score(X_train, y_train), lg.score(X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth = 7)
tree.fit(X_train, y_train)
tree.score(X_train, y_train), tree.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state = 0, max_depth = 5)
forest.fit(X_train, y_train)
forest.score(X_train, y_train), forest.score(X_test, y_test)

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state = 0)
xgb.fit(X_train, y_train)
xgb.score(X_train, y_train), xgb.score(X_test, y_test)